In [10]:
import pandas as pd
import numpy as np
import pathlib
import calendar
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [11]:
from sqlalchemy import create_engine

database_filename = 'tdpdata.db'
table_name = 'tdpsheet'
engine = create_engine('sqlite:///{}'.format(database_filename)) 

In [12]:
df_sql_test = pd.read_sql(table_name, engine, parse_dates='OBSERVATION_TIME')

In [13]:
df_sql_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5754574 entries, 0 to 5754573
Data columns (total 24 columns):
 #   Column            Dtype              
---  ------            -----              
 0   SITE_NUMBER       object             
 1   OBSERVATION_TIME  datetime64[ns, UTC]
 2   REF_TEMP          float64            
 3   AMBIENT_AIR_TEMP  float64            
 4   IN_PAVEMENT_TEMP  float64            
 5   INTERNAL_TEMP     float64            
 6   BATTERY_VOLTAGE   float64            
 7   TMR_PAV           float64            
 8   TMR_SUB_0         float64            
 9   TMR_SUB_3         float64            
 10  TMR_SUB_6         float64            
 11  TMR_SUB_9         float64            
 12  TMR_SUB_12        float64            
 13  TMR_SUB_18        float64            
 14  TMR_SUB_24        float64            
 15  TMR_SUB_30        float64            
 16  TMR_SUB_36        float64            
 17  TMR_SUB_42        float64            
 18  TMR_SUB_48        floa

In [14]:
df_sql_test.isnull().sum()

SITE_NUMBER         0
OBSERVATION_TIME    0
REF_TEMP            0
AMBIENT_AIR_TEMP    0
IN_PAVEMENT_TEMP    0
INTERNAL_TEMP       0
BATTERY_VOLTAGE     0
TMR_PAV             0
TMR_SUB_0           0
TMR_SUB_3           0
TMR_SUB_6           0
TMR_SUB_9           0
TMR_SUB_12          0
TMR_SUB_18          0
TMR_SUB_24          0
TMR_SUB_30          0
TMR_SUB_36          0
TMR_SUB_42          0
TMR_SUB_48          0
TMR_SUB_54          0
TMR_SUB_60          0
TMR_SUB_66          0
TMR_SUB_72          0
TIMEZONE_FLAG       0
dtype: int64

In [16]:
df = df.drop(['REF_TEMP', 
                  'IN_PAVEMENT_TEMP', 
                  'INTERNAL_TEMP', 
                  'BATTERY_VOLTAGE', 
                  'TIMEZONE_FLAG'], axis=1)

In [9]:
def replace_error_code_values(df):  
    """
    replace all error values with np.nan or NaN 
    (pandas/numpy's "null")
    """
    import numpy as np
    df.replace(to_replace=['-9999.000000', 
                             '-9999.0', 
                             '-9999.00',
                             '-9999.000', 
                             -9999.00, 
                             -9999.000000, 
                             -9999.0,
                             -6999.0,
                             -6999.00,
                             -6999.000,
                             -6999.000000,
                             -9999.000], value=np.nan, inplace=True)
    
    print(df.isnull().sum())
    return df

In [15]:
df = replace_error_code_values(df_sql_test)

SITE_NUMBER               0
OBSERVATION_TIME          0
REF_TEMP            2540578
AMBIENT_AIR_TEMP     435457
IN_PAVEMENT_TEMP    1248200
INTERNAL_TEMP       1866183
BATTERY_VOLTAGE     1860473
TMR_PAV              176886
TMR_SUB_0             95139
TMR_SUB_3            100901
TMR_SUB_6            100903
TMR_SUB_9            100903
TMR_SUB_12           100904
TMR_SUB_18           100906
TMR_SUB_24           100912
TMR_SUB_30           100918
TMR_SUB_36           100920
TMR_SUB_42           100926
TMR_SUB_48           100932
TMR_SUB_54           100934
TMR_SUB_60           100942
TMR_SUB_66           100949
TMR_SUB_72           188392
TIMEZONE_FLAG             0
dtype: int64


In [17]:
def remove_duplicate_adjacent_values(df):
    
    print(df.shape)
    
    print(df.isnull().sum())

    # isolate sensor depth columns
    sensors_df = df.iloc[:, 2:]
    
    """# isolate rows that have duplicates between columns
    sensor_dup = sensors_df[np.logical_or(sensors_df.diff(axis=1) == 0, 
                                          sensors_df.diff(axis=1, periods=-1) == 0)]
    
    sensor_dup.shape[0]
    
    sensor_dup_clean = sensor_dup.dropna()
    sensor_dup_clean.shape[0]"""
    
    sensors_df[np.logical_or(sensors_df.diff(axis=1) == 0, 
                             sensors_df.diff(axis=1, periods=-1) == 0)] = np.nan
    
    cols = df.iloc[:, :2]
    
    df = sensors_df.merge(cols, left_index=True, right_index=True)
    
    df_cols = df.columns.tolist()
    
    df_cols = df_cols[-2:] + df_cols[:-2]
    
    df = df[df_cols]
    
    print(df.isnull().sum())
    
    return df

In [18]:
df = remove_duplicate_adjacent_values(df)

(5754574, 19)
SITE_NUMBER              0
OBSERVATION_TIME         0
AMBIENT_AIR_TEMP    435457
TMR_PAV             176886
TMR_SUB_0            95139
TMR_SUB_3           100901
TMR_SUB_6           100903
TMR_SUB_9           100903
TMR_SUB_12          100904
TMR_SUB_18          100906
TMR_SUB_24          100912
TMR_SUB_30          100918
TMR_SUB_36          100920
TMR_SUB_42          100926
TMR_SUB_48          100932
TMR_SUB_54          100934
TMR_SUB_60          100942
TMR_SUB_66          100949
TMR_SUB_72          188392
dtype: int64
SITE_NUMBER              0
OBSERVATION_TIME         0
AMBIENT_AIR_TEMP    452782
TMR_PAV             493515
TMR_SUB_0           687613
TMR_SUB_3           776165
TMR_SUB_6           828394
TMR_SUB_9           863766
TMR_SUB_12          798668
TMR_SUB_18          725839
TMR_SUB_24          730754
TMR_SUB_30          719449
TMR_SUB_36          710378
TMR_SUB_42          707144
TMR_SUB_48          708935
TMR_SUB_54          721703
TMR_SUB_60          763330
T

In [19]:
def clean_extreme_values(df):
    
    # next comment is secondary attempt that uses masking, but could not get to function
    # data['TMR_PAV'] = np.where(data.loc[(data['TMR_PAV'] >= 150) | (data['TMR_PAV'] <= -50)],np.nan,data['TMR_PAV']
    # TMR_PAV
    df['TMR_PAV'] = np.where((df['TMR_PAV'] >= 110) | (df['TMR_PAV'] <= -30),np.nan,df['TMR_PAV'])  
    
    # AMBIENT_AIR_TEMP
    df['AMBIENT_AIR_TEMP'] = np.where((df['AMBIENT_AIR_TEMP'] >= 105) | (df['AMBIENT_AIR_TEMP'] <= -55),
                                        np.nan,df['AMBIENT_AIR_TEMP'])
    
    # IN_PAVEMENT_TEMP (Only if included)
    # data['IN_PAVEMENT_TEMP'] = np.where((data['IN_PAVEMENT_TEMP'] >= 110) | (data['IN_PAVEMENT_TEMP'] <= -30),
    #                                     np.nan,data['IN_PAVEMENT_TEMP'])
    
    # TMR_SUB_0
    df['TMR_SUB_0'] = np.where((df['TMR_SUB_0'] >= 110) | (df['TMR_SUB_0'] <= -25),
                                        np.nan,df['TMR_SUB_0'])
    
    # TMR_SUB_3
    df['TMR_SUB_3'] = np.where((df['TMR_SUB_3'] >= 100) | (df['TMR_SUB_3'] <= -20),
                                        np.nan,df['TMR_SUB_3'])
    
    # TMR_SUB_6
    df['TMR_SUB_6'] = np.where((df['TMR_SUB_6'] >= 95) | (df['TMR_SUB_6'] <= -15),
                                        np.nan,df['TMR_SUB_6'])
    
    # TMR_SUB_9 
    df['TMR_SUB_9'] = np.where((df['TMR_SUB_9'] >= 90) | (df['TMR_SUB_9'] <= -15),
                                        np.nan,df['TMR_SUB_9'])
    
    # TMR_SUB_12
    df['TMR_SUB_12'] = np.where((df['TMR_SUB_12'] >= 85) | (df['TMR_SUB_12'] <= -10),
                                        np.nan,df['TMR_SUB_12'])
    
    # TMR_SUB_18
    df['TMR_SUB_18'] = np.where((df['TMR_SUB_18'] >= 85) | (df['TMR_SUB_18'] <= -10),
                                        np.nan,df['TMR_SUB_18'])
    
    # TMR_SUB_24
    df['TMR_SUB_24'] = np.where((df['TMR_SUB_24'] >= 85) | (df['TMR_SUB_24'] <= -10),
                                        np.nan,df['TMR_SUB_24'])
    
    # TMR_SUB_30
    df['TMR_SUB_30'] = np.where((df['TMR_SUB_30'] >= 80) | (df['TMR_SUB_30'] <= -5),
                                        np.nan,df['TMR_SUB_30'])
    
    # TMR_SUB_36
    df['TMR_SUB_36'] = np.where((df['TMR_SUB_36'] >= 80) | (df['TMR_SUB_36'] <= -5),
                                        np.nan,df['TMR_SUB_36'])
    
    # TMR_SUB_42
    df['TMR_SUB_42'] = np.where((df['TMR_SUB_42'] >= 75) | (df['TMR_SUB_42'] <= 0),
                                        np.nan,df['TMR_SUB_42'])
    
    # TMR_SUB_48
    df['TMR_SUB_48'] = np.where((df['TMR_SUB_48'] >= 70) | (df['TMR_SUB_48'] <= 0),
                                        np.nan,df['TMR_SUB_48'])
    
    # TMR_SUB_54
    df['TMR_SUB_54'] = np.where((df['TMR_SUB_54'] >= 70) | (df['TMR_SUB_54'] <= 5),
                                        np.nan,df['TMR_SUB_54'])
    
    # TMR_SUB_60
    df['TMR_SUB_60'] = np.where((df['TMR_SUB_60'] >= 70) | (df['TMR_SUB_60'] <= 5),
                                        np.nan,df['TMR_SUB_60'])
    
    # TMR_SUB_66
    df['TMR_SUB_66'] = np.where((df['TMR_SUB_66'] >= 65) | (df['TMR_SUB_66'] <= 10),
                                        np.nan,df['TMR_SUB_66'])
    
    # TMR_SUB_72
    df['TMR_SUB_72'] = np.where((df['TMR_SUB_72'] >= 65) | (df['TMR_SUB_72'] <= 10),
                                        np.nan,df['TMR_SUB_72'])
    
    return df

In [20]:
df = clean_extreme_values(df)

In [21]:
df = df.loc[df['SITE_NUMBER']=='128']

df = df[['SITE_NUMBER','OBSERVATION_TIME','TMR_SUB_18']]

df.head()

,SITE_NUMBER,OBSERVATION_TIME,TMR_SUB_18
84994,128,2003-12-14 17:00:00+00:00,NaN
84995,128,2003-12-14 18:00:00+00:00,30.10
84996,128,2003-12-14 19:00:00+00:00,29.88
84997,128,2003-12-14 20:00:00+00:00,29.88
84998,128,2003-12-14 21:00:00+00:00,NaN


In [22]:
df.isnull().sum()

SITE_NUMBER            0
OBSERVATION_TIME       0
TMR_SUB_18          3916
dtype: int64

In [24]:
df.shape

(110581, 3)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110581 entries, 84994 to 5751771
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype              
---  ------            --------------   -----              
 0   SITE_NUMBER       110581 non-null  object             
 1   OBSERVATION_TIME  110581 non-null  datetime64[ns, UTC]
 2   TMR_SUB_18        106665 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), object(1)
memory usage: 3.4+ MB


In [26]:
df['year'] = df.OBSERVATION_TIME.dt.year
df['month'] = df.OBSERVATION_TIME.dt.month

df['day'] = df.OBSERVATION_TIME.dt.day
df['hour'] = df.OBSERVATION_TIME.dt.hour

In [27]:
df.head()

,SITE_NUMBER,OBSERVATION_TIME,TMR_SUB_18,year,month,day,hour
84994,128,2003-12-14 17:00:00+00:00,NaN,2003,12,14,17
84995,128,2003-12-14 18:00:00+00:00,30.10,2003,12,14,18
84996,128,2003-12-14 19:00:00+00:00,29.88,2003,12,14,19
84997,128,2003-12-14 20:00:00+00:00,29.88,2003,12,14,20
84998,128,2003-12-14 21:00:00+00:00,NaN,2003,12,14,21


In [29]:
# filename = "df_128_all_18.pkl"
filename = "df_128_all_years_18_clean.pkl"
df.to_pickle(filename)

In [34]:
df.groupby(['year','month','day'])['TMR_SUB_18'].count() ==24

year  month  day
2003  1      31     False
      2      1      False
             2      False
             3      False
             4      False
             5      False
             6      False
             7      False
             8      False
             9      False
             10     False
             11     False
             12     False
             13     False
             14     False
             15     False
             16     False
             17     False
             18     False
             19      True
             20      True
             21      True
             22      True
             23     False
             24     False
             25     False
             26     False
             27     False
             28     False
      3      1      False
             2      False
             3      False
             4      False
             5      False
             6      False
             7      False
             8       True
             9       